# CryptoBot with Binance Project

## Import Libraries

In [1]:
# import libraries
import config
import pandas as pd
from binance.client import Client
from pymongo import MongoClient


## Get Data for "BTCUSDT" with API

In [2]:
client = Client(config.APIKey, config.SecretKey)

klines_BTCUSDT = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1DAY, "1 Jan, 2017", "now")

df = pd.DataFrame(klines_BTCUSDT, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')

df.tail()

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
2089,2023-05-07,28848.19000000,29138.29000000,28395.23000000,28430.10000000,30003.41028000,2023-05-07 23:59:59.999,865838535.32638210,855425,14904.01102000,430162358.62942520,0
2090,2023-05-08,28430.09000000,28631.01000000,27262.00000000,27668.79000000,68244.36179000,2023-05-08 23:59:59.999,1903052694.66841270,1600637,32347.56935000,901988104.11036000,0
2091,2023-05-09,27668.80000000,27818.00000000,27353.00000000,27628.27000000,40113.31069000,2023-05-09 23:59:59.999,1106923908.30584690,1003698,20094.09737000,554561922.80537130,0
2092,2023-05-10,27628.28000000,28331.42000000,26777.00000000,27598.75000000,71155.11355000,2023-05-10 23:59:59.999,1971173412.43550200,1534579,34398.83676000,953153016.50961890,0
2093,2023-05-11,27598.74000000,27630.14000000,26702.05000000,26985.55000000,44129.43033000,2023-05-11 23:59:59.999,1201244992.56405450,1143700,21564.25247000,586914203.81729590,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2094 entries, 0 to 2093
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   timestamp                     2094 non-null   datetime64[ns]
 1   open                          2094 non-null   object        
 2   high                          2094 non-null   object        
 3   low                           2094 non-null   object        
 4   close                         2094 non-null   object        
 5   volume                        2094 non-null   object        
 6   close_time                    2094 non-null   datetime64[ns]
 7   quote_asset_volume            2094 non-null   object        
 8   number_of_trades              2094 non-null   int64         
 9   taker_buy_base_asset_volume   2094 non-null   object        
 10  taker_buy_quote_asset_volume  2094 non-null   object        
 11  ignore                        

## Change data types object to float 

In [4]:
df.columns

Index(['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time',
       'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume',
       'taker_buy_quote_asset_volume', 'ignore'],
      dtype='object')

In [5]:
cols = ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'taker_buy_base_asset_volume','taker_buy_quote_asset_volume', 'ignore']

df[cols] = df[cols].astype(float)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2094 entries, 0 to 2093
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   timestamp                     2094 non-null   datetime64[ns]
 1   open                          2094 non-null   float64       
 2   high                          2094 non-null   float64       
 3   low                           2094 non-null   float64       
 4   close                         2094 non-null   float64       
 5   volume                        2094 non-null   float64       
 6   close_time                    2094 non-null   datetime64[ns]
 7   quote_asset_volume            2094 non-null   float64       
 8   number_of_trades              2094 non-null   int64         
 9   taker_buy_base_asset_volume   2094 non-null   float64       
 10  taker_buy_quote_asset_volume  2094 non-null   float64       
 11  ignore                        

## Save data to csv

In [6]:
df.to_csv("data/BTCUSDT_2017_01_01_to_2023_05_11.csv", index=False)

## MongoDB Connection

In [11]:
from pymongo import MongoClient

# Create MongoDB client
client = MongoClient('localhost', 27017)

# Create database and collection
binance = client['binance']

btc = binance['btc']

# Convert df (Dataframe) to dictionary
data = df.to_dict(orient='records')

# Add all datas to MongoDB 
btc.insert_many(data)

# See all databases
client.list_database_names()


['admin', 'binance', 'config', 'local', 'sample']

In [12]:
btc.find_one()

{'_id': ObjectId('645d578e03834d5ed5a1c3bb'),
 'timestamp': datetime.datetime(2017, 8, 17, 0, 0),
 'open': 4261.48,
 'high': 4485.39,
 'low': 4200.74,
 'close': 4285.08,
 'volume': 795.150377,
 'close_time': datetime.datetime(2017, 8, 17, 23, 59, 59, 999000),
 'quote_asset_volume': 3454770.05073206,
 'number_of_trades': 3427,
 'taker_buy_base_asset_volume': 616.248541,
 'taker_buy_quote_asset_volume': 2678216.40060401,
 'ignore': 0.0}